In [3]:
import os
import cv2

print("Initial Load cell: ", end='')

fileList = os.listdir("TestSet")

fileList = [f"TestSet/{i}" for i in fileList]
print(f"{len(fileList)} images loaded from folder")

imageArray = [cv2.imread(i) for i in fileList]

print(''.join(['-' for i in range(50)]))


Initial Load cell: 88 images loaded from folder
--------------------------------------------------


In [4]:
# Reliable lane markings

# https://github.com/galenballew/SDC-Lane-and-Vehicle-Detection-Tracking
# OMG FOUND A THING!

import cv2
from tqdm import tqdm
from IPython.display import clear_output
import numpy as np

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)

    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    #filling pixels inside the polygon defined by "vertices" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

showHistogram = False
showCanny = False
showRoi = False
timeDelay = 30

reliableMarkings = []
print("Reliable Lane markings cell: ")
for im in tqdm(imageArray):
    # Feature extraction based off https://medium.com/@galen.ballew/opencv-lanedetection-419361364fc0
    # Only white lines are going to be looked out, so only one threshold is needed
    grayIm = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)  # White uses grayscale threshold
    grayImHist = cv2.equalizeHist(grayIm)  # Equalize the image to account for changes in exposure
    
    if showHistogram:
        cv2.imshow("PreHist", cv2.resize(grayIm, None, fx=0.5, fy=0.5))
        cv2.imshow("PostHist", cv2.resize(grayImHist, None, fx=0.5, fy=0.5))
        cv2.waitKey(timeDelay)
    
    # Apply a mask to find the white lines on the road
    mask_white = cv2.inRange(grayImHist, 200, 255)  # Threshold bright values
    kernel_size = 5
    mask_white = cv2.GaussianBlur(mask_white,(kernel_size,kernel_size),0)
    mask_white = cv2.medianBlur(mask_white, 11)
    
    # Apply canny edge detection to look for lines with relevant gradients
    low_threshold = 50
    high_threshold = 150
    canny_edges = cv2.Canny(mask_white, low_threshold, high_threshold)
    if showCanny:
        cv2.imshow("Canny", cv2.resize(canny_edges, None, fx=0.5, fy=0.5))
        cv2.waitKey(timeDelay)
    # Apply a region of interest
    imshape = im.shape
    lower_left = [imshape[1]/10,imshape[0]]
    lower_right = [imshape[1]-imshape[1]/10,imshape[0]]
    top_left = [imshape[1]/2-imshape[1]/8,imshape[0]/2+imshape[0]/15]
    top_right = [imshape[1]/2+imshape[1]/8,imshape[0]/2+imshape[0]/15]
    vertices = [np.array([lower_left,top_left,top_right,lower_right],dtype=np.int32)]  # Get the relevant roi
    
    roi_image = region_of_interest(canny_edges, vertices)
    # Apply the roi
    if showRoi:
        cv2.imshow("Roi", cv2.resize(roi_image, None, fx=0.5, fy=0.5))
        cv2.waitKey(timeDelay)
        
    reliableMarkings.append(roi_image)
    clear_output(True)


print(''.join(['-' for i in range(50)]))

100%|██████████| 88/88 [00:12<00:00,  7.32it/s]


--------------------------------------------------


In [8]:
# Show of all the processing types
import cv2
from matplotlib import pyplot as plt
from IPython.display import clear_output
from tqdm import tqdm

fig = plt.figure()
ims = []
for img, reliable in tqdm(zip(imageArray, reliableMarkings)):
    
    cv2.imshow("Original", cv2.resize(img, None, fx=0.5, fy=0.5))
    cv2.imshow("Reliable markings", cv2.resize(reliable, None, fx=0.5, fy=0.5))
    cv2.waitKey(100)
    clear_output(True)

print(''.join(['-' for i in range(50)]))




88it [00:09,  9.31it/s]


--------------------------------------------------


<Figure size 432x288 with 0 Axes>